# Search and Evaluate

In [1]:
import pandas as pd

annotated = pd.read_excel(r"D:\66 CausalMap\Panasonic-IDS\data\MarketIntelligenceReport_Annotation_Tasks.xlsx", engine='openpyxl', sheet_name='MarketIntelligenceReport_Task1')
data = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MIR.csv")

In [2]:
df = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MarketIntelligenceReport.csv")
df['row_id'] = df.index
cols = ['Translated']

# duplicates, except first
m1 = df.duplicated(cols)
# duplicates including first
m2 = df.duplicated(cols, keep=False)

df['first_occurrence'] = (
 df[cols].merge(df.loc[~m1, cols+['row_id']],how='left')['row_id'].astype(str).where(m1, m2)
)
df

,id,Search date,Geography,Title,Text,Publication date,Original,Translated,Month,Year,Label,row_id,first_occurrence
0,1,27/9/2022 15:22,global,What Makes India The Deathbed Of Global Automa...,"Prior to Ford, two more American firms- Genera...",10/1/2022,en,"Prior to Ford, two more American firms- Genera...",Jan,2022,4,0,False
1,2,27/9/2022 15:22,global,Ford to stop making cars in India,The U.S. carmaker entered India 25 years ago b...,9/9/2021,en,The U.S. carmaker entered India 25 years ago b...,Sep,2021,1,1,True
2,3,27/9/2022 15:22,global,"Ford Motor Company bails on India, will shut c...",Ford is shutting its car factories in India af...,9/9/2021,en,Ford is shutting its car factories in India af...,Sep,2021,1,2,True
3,4,27/9/2022 15:22,global,India approves $3.5 bln scheme to boost clean ...,The government will give about 260 billion rup...,15/9/2021,en,The government will give about 260 billion rup...,Sep,2021,1,3,False
4,5,27/9/2022 15:22,global,More electric vehicles in India soon as automa...,More electric vehicles in India soon as automa...,20/10/2021,en,More electric vehicles in India soon as automa...,Oct,2021,4,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9121,146,22/9/2022 12:27,global,Kebijakan Ketat Covid-19 Bikin Pengusaha Eropa...,"Liputan6.com, Jakarta - Para pengusaha dari Er...",21/9/2022,id,"Liputan6.com, Jakarta - Europeans from Europe ...",Sep,2022,1,9121,True
9122,522,22/9/2022 13:18,china,Kebijakan Ketat Covid-19 Bikin Pengusaha Eropa...,"Liputan6.com, Jakarta - Para pengusaha dari Er...",21/9/2022,id,"Liputan6.com, Jakarta - Europeans from Europe ...",Sep,2022,1,9122,9121
9123,549,22/9/2022 15:37,japan,Certificación EV SSL Participación en el tamañ...,Solicite una copia de muestra del informe:http...,21/9/2022,es,Request a copy of the report: https: //www.mar...,Sep,2022,1,9123,False
9124,550,22/9/2022 15:37,japan,Sistema de gestión térmica de batería EV Dinám...,El informe de investigación de mercado de Sist...,21/9/2022,es,The EV Battery Thermal Management System Marke...,Sep,2022,4,9124,False


In [3]:
import pickle

file = open(r"D:\66 CausalMap\Panasonic-IDS\outs\20230313_MIR_rels.obj",'rb')
f_infos = pickle.load(file)
file.close()

In [4]:
from collections import defaultdict

fid2docid = defaultdict(list)
for k,v in f_infos.items():
    for vv in v:
        fid2docid[vv['doc_id']].append(k)

In [9]:
row_id = 1227
rowid2docid = {v:k for k,v in enumerate(data['row_id'])}
rowid2docid[row_id]

KeyError: 1227

In [11]:
def print_extracted_rels_by_row_id(row_id):
    if row_id not in rowid2docid:
        old = row_id
        row_id = int(df[df['row_id']==row_id]['first_occurrence'].values[0])
        print(f'convert {old}-->{row_id}')
    
    doc_id = rowid2docid[row_id]
    for k in fid2docid[doc_id]:
        print(f_infos[k])
    
    return len(fid2docid[doc_id]) if len(fid2docid[doc_id]) is not None else 0
        
print_extracted_rels_by_row_id(row_id=1226)

[{'text': 'RIYADH: China stocks rose on Monday after the governor of the country’s central bank vowed to increase the implementation of prudent monetary policy to support the real economy.', 'cause_span': 'the governor of the country', 'effect_span': 'to increase the implementation of p rud ent monetary policy', 'doc_id': 1014, 'sent_id': 0, 'method': ['unicausalm'], 'causenet_patterns': None}]
[{'text': 'RIYADH: China stocks rose on Monday after the governor of the country’s central bank vowed to increase the implementation of prudent monetary policy to support the real economy.', 'cause_span': 'to support the real economy .', 'effect_span': 'to increase the implementation of p rud ent monetary policy', 'doc_id': 1014, 'sent_id': 0, 'method': ['unicausalm'], 'causenet_patterns': None}]
[{'text': 'RIYADH: China stocks rose on Monday after the governor of the country’s central bank vowed to increase the implementation of prudent monetary policy to support the real economy.', 'cause_span

4

In [7]:
counts = 0
for row_id in annotated['row_id'].unique():
    counts+=print_extracted_rels_by_row_id(row_id)
counts

convert 235-->145
convert 292-->95


0

# Format Predictions
- into CNC "ref" format
- into CNC predictions format

In [2]:
import pickle
import re, os
import pandas as pd
from collections import defaultdict

file = open(r"D:\66 CausalMap\Panasonic-IDS\outs\20230313_MIR_rels.obj",'rb')
f_infos = pickle.load(file)
file.close()
fid2docid = defaultdict(list)
for k,v in f_infos.items():
    for vv in v:
        fid2docid[vv['doc_id']].append(k)
        
annotated_df = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MIR_annotated_grouped.csv")
annotated_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents,token_overlap,Unnamed: 12,Unnamed: 13,Unnamed: 14,causal_text_w_pairs,num_rs
0,MIR,51,1,0,MIR_51_1_0,"From to Uber and , many foreign companies have...","<ARG1>From to Uber and , many foreign companie...",1,1,NaN,1,1,1,True,NaN,"['<ARG1>From to Uber and , many foreign compan...",1
1,MIR,51,3,0,MIR_51_3_0,Foreign or local electric cars (EVs) will grow...,<ARG1>Foreign or local electric cars (EVs) wil...,1,1,NaN,1,1,1,True,NaN,['<ARG1>Foreign or local electric cars (EVs) w...,1
2,MIR,51,6,0,MIR_51_6_0,"""Given these conditions, it is understandable ...","<ARG0>""Given these conditions</ARG0>, it is un...",1,1,NaN,1,1,1,True,NaN,"['<ARG0>""Given these conditions</ARG0>, it is ...",1
3,MIR,78,0,0,MIR_78_0_0,Toyota in India has largely pivoted toward hyb...,<ARG1>Toyota in India has largely pivoted towa...,1,1,NaN,1,1,1,True,NaN,['<ARG1>Toyota in India has largely pivoted to...,1
4,MIR,78,2,0,MIR_78_2_0,The government keeps taxes on cars and motorbi...,<ARG0>The government keeps taxes on cars and m...,1,1,NaN,1,1,1,True,NaN,['<ARG0>The government keeps taxes on cars and...,1


In [3]:
def find_substring_in_text(text, substring):
    # replace [UNK] with specials
    substring = re.sub('\[UNK\]','\u200b',substring)
    # remove all spaces
    substring = re.sub(' ','',substring)    
    # insert optional spaces
    new_substring = ''
    for i in substring:
        # remember to escape chars like () \ etc.
        new_substring+=re.escape(i)+'\s*'
    new_substring = re.sub('\u200b','.*',new_substring)
    search = re.search(new_substring,text)
    
    if search is None:
        raise ValueError(f"{text}\n{substring}")
    
    return search.start(), search.end()

# text = 'At the wind tunnel complex in Auburn Hills, Michigan, construction is in progress to install moving ground plane technology (rolling road), which simulates on-road travel while allowing test vehicles to remain static.'
# effect = 'moving ground plane technology ( rolling road )'

# text = "Chicken Plus was rated the best tasting frozen chicken nugget in the US with just 50% real chicken in it – imagine if all meat products also opted for hybrid blending, this would solve so many problems including health and sustainability challenges,”​ Better Meat Co Co-Founder and CEO Paul Shapiro told us."
# substring = ",thiswouldsolvesomanyproblemsincludinghealthandsustainabilitychallenges,”BetterMeatCoCo-FounderandCEOPaulShapirotoldus."

text = "The defendants and their Hainan State Security Department (HSSD) conspirators sought to obfuscate the Chinese government’s role in such theft by establishing a front company, Hainan Xiandun Technology Development Co. , Ltd. (海南仙盾) (Hainan Xiandun), since disbanded, to operate out of Haikou, Hainan Province."
substring = "establishingafrontcompany,HainanXiandunTechnologyDevelopmentCo.,Ltd.(海南[UNK][UNK])(HainanXiandun),sincedisbanded,"
find_substring_in_text(text.replace('\u200b', ''), substring.replace('\u200b', ''))

(145, 266)

In [4]:
cols = ['corpus','doc_id','sent_id','eg_id','index','text','text_w_pairs','seq_label','pair_label','context','num_sents','method']
pred = []
prev_id = None
eg_id = 0

for doc_id in annotated_df['doc_id'].unique():
    
    for cause,effect in fid2docid[doc_id]:
        
        for ddict in f_infos[(cause,effect)]:
            
            if ddict['doc_id']!=doc_id:
                continue
            
            sent_id = ddict['sent_id']
            curr_id = f'MIR_{doc_id}_{sent_id}'
            if curr_id==prev_id:
                eg_id+=1
            else:
                eg_id=0
            prev_id = curr_id

            text = ddict['text']
            cause = ddict['cause_span'].strip()
            effect = ddict['effect_span'].strip()
            
            cause_start, cause_end = find_substring_in_text(text, cause)
            effect_start, effect_end = find_substring_in_text(text, effect)
            
            adjustment = 13
            text_w_pairs = text
            if cause_start<effect_start:
                effect_start+=adjustment
                effect_end+=adjustment
                text_w_pairs = text_w_pairs[:cause_start] + '<ARG0>' + text_w_pairs[cause_start:cause_end] + '</ARG0>'+ text_w_pairs[cause_end:]
                text_w_pairs = text_w_pairs[:effect_start] + '<ARG1>' + text_w_pairs[effect_start:effect_end] + '</ARG1>'+ text_w_pairs[effect_end:]
            else:
                cause_start+=adjustment
                cause_end+=adjustment
                text_w_pairs = text_w_pairs[:effect_start] + '<ARG1>' + text_w_pairs[effect_start:effect_end] + '</ARG1>'+ text_w_pairs[effect_end:]
                text_w_pairs = text_w_pairs[:cause_start] + '<ARG0>' + text_w_pairs[cause_start:cause_end] + '</ARG0>'+ text_w_pairs[cause_end:]
            
            if ('<ARG0>' not in text_w_pairs) or ('<ARG1>' not in text_w_pairs):
                raise ValueError(f'Cause/Effect Arg not found: \n{text_w_pairs}')

            pred.append([
                'MIR',
                doc_id,
                sent_id,
                eg_id,
                f'{curr_id}_{eg_id}',
                text,
                text_w_pairs,
                1,
                1,
                '',
                1,
                '&'.join(ddict['method'])
            ])
        
        
pred = pd.DataFrame(pred, columns=cols)
pred

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents,method
0,MIR,51,1,0,MIR_51_1_0,"From to Uber and , many foreign companies have...","From to Uber <ARG1>and , many foreign companie...",1,1,,1,unicausal+
1,MIR,51,2,0,MIR_51_2_0,It would be hard for Tesla to achieve mass-mar...,<ARG0>It would be hard for Tesla to </ARG0>ach...,1,1,,1,unicausal
2,MIR,51,3,0,MIR_51_3_0,Foreign or local electric cars (EVs) will grow...,<ARG1>Foreign or local electric cars (EVs) wil...,1,1,,1,unicausal
3,MIR,51,6,0,MIR_51_6_0,"""Given these conditions, it is understandable ...","""Given <ARG0>these conditions</ARG0><ARG1>, it...",1,1,,1,unicausal
4,MIR,78,0,0,MIR_78_0_0,Toyota in India has largely pivoted toward hyb...,Toyota in India has largely pivoted toward hyb...,1,1,,1,unicausal
5,MIR,78,2,0,MIR_78_2_0,The government keeps taxes on cars and motorbi...,<ARG0>The government keeps taxes on cars and m...,1,1,,1,unicausal
6,MIR,78,8,0,MIR_78_8_0,"Elon Musk, the billionaire founder of Tesla In...","Elon Musk, the billionaire founder of Tesla In...",1,1,,1,unicausal+
7,MIR,94,1,0,MIR_94_1_0,The report’s focus solely on electric vehicles...,The report’s focus solely on electric vehicles...,1,1,,1,unicausal+
8,MIR,94,6,0,MIR_94_6_0,"To kick-start the shift, the report suggests b...","To kick-start the shift, <ARG1>the report sugg...",1,1,,1,unicausalm
9,MIR,94,7,0,MIR_94_7_0,It also recommends setting up battery swapping...,It also recommends <ARG1>setting up battery sw...,1,1,,1,unicausal


In [5]:
identifier_cols = ['corpus','doc_id','sent_id']

data2 = pred.copy()
grouped_causal_data = data2.pivot_table(
    index=identifier_cols+['text'], 
    columns='eg_id',
    aggfunc=lambda x: ' '.join(x), 
    values='text_w_pairs', 
    fill_value=0
).reset_index()

eg_id_cols = [c for c in grouped_causal_data.columns if c not in identifier_cols+['text']]
grouped_causal_data['causal_text_w_pairs'] = grouped_causal_data[eg_id_cols].values.tolist()
grouped_causal_data['eg_id'] = grouped_causal_data['causal_text_w_pairs'].apply(lambda x_list: min([i for i,x in enumerate(x_list) if x!=0]))
grouped_causal_data['causal_text_w_pairs'] = grouped_causal_data['causal_text_w_pairs'].apply(lambda x_list: [x for x in x_list if x!=0 and x!=""])
grouped_causal_data['num_rs'] = grouped_causal_data['causal_text_w_pairs'].apply(lambda x_list: int(len(x_list)))
grouped_causal_data = grouped_causal_data.drop(columns=eg_id_cols+['text'])

data2 = pred.copy()
grouped_causal_data2 = data2.pivot_table(
    index=identifier_cols+['text'], 
    columns='eg_id',
    aggfunc=lambda x: ' '.join(x), 
    values='method', 
    fill_value=0
).reset_index()

grouped_causal_data['method'] = grouped_causal_data2[eg_id_cols].values.tolist()
grouped_causal_data['method'] = grouped_causal_data['method'].apply(lambda x_list: [x for x in x_list if x!=0 and x!=""])
del(grouped_causal_data2)
grouped_causal_data

eg_id,corpus,doc_id,sent_id,causal_text_w_pairs,eg_id,num_rs,method
0,MIR,51,1,"[From to Uber <ARG1>and , many foreign compani...",0,1,[unicausal+]
1,MIR,51,2,[<ARG0>It would be hard for Tesla to </ARG0>ac...,0,1,[unicausal]
2,MIR,51,3,[<ARG1>Foreign or local electric cars (EVs) wi...,0,1,[unicausal]
3,MIR,51,6,"[""Given <ARG0>these conditions</ARG0><ARG1>, i...",0,1,[unicausal]
4,MIR,78,0,[Toyota in India has largely pivoted toward hy...,0,1,[unicausal]
5,MIR,78,2,[<ARG0>The government keeps taxes on cars and ...,0,1,[unicausal]
6,MIR,78,8,"[Elon Musk, the billionaire founder of Tesla I...",0,1,[unicausal+]
7,MIR,94,1,[The report’s focus solely on electric vehicle...,0,1,[unicausal+]
8,MIR,94,6,"[To kick-start the shift, <ARG1>the report sug...",0,1,[unicausalm]
9,MIR,94,7,[It also recommends <ARG1>setting up battery s...,0,1,[unicausal]


In [6]:
grouped_pred = pred.drop(columns='method').merge(grouped_causal_data, how='right', on=identifier_cols+['eg_id'])
grouped_pred['num_rs'] = grouped_pred['num_rs'].fillna(0)
grouped_pred['num_rs'] = grouped_pred['num_rs'].astype(int)
grouped_pred

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents,causal_text_w_pairs,num_rs,method
0,MIR,51,1,0,MIR_51_1_0,"From to Uber and , many foreign companies have...","From to Uber <ARG1>and , many foreign companie...",1,1,,1,"[From to Uber <ARG1>and , many foreign compani...",1,[unicausal+]
1,MIR,51,2,0,MIR_51_2_0,It would be hard for Tesla to achieve mass-mar...,<ARG0>It would be hard for Tesla to </ARG0>ach...,1,1,,1,[<ARG0>It would be hard for Tesla to </ARG0>ac...,1,[unicausal]
2,MIR,51,3,0,MIR_51_3_0,Foreign or local electric cars (EVs) will grow...,<ARG1>Foreign or local electric cars (EVs) wil...,1,1,,1,[<ARG1>Foreign or local electric cars (EVs) wi...,1,[unicausal]
3,MIR,51,6,0,MIR_51_6_0,"""Given these conditions, it is understandable ...","""Given <ARG0>these conditions</ARG0><ARG1>, it...",1,1,,1,"[""Given <ARG0>these conditions</ARG0><ARG1>, i...",1,[unicausal]
4,MIR,78,0,0,MIR_78_0_0,Toyota in India has largely pivoted toward hyb...,Toyota in India has largely pivoted toward hyb...,1,1,,1,[Toyota in India has largely pivoted toward hy...,1,[unicausal]
5,MIR,78,2,0,MIR_78_2_0,The government keeps taxes on cars and motorbi...,<ARG0>The government keeps taxes on cars and m...,1,1,,1,[<ARG0>The government keeps taxes on cars and ...,1,[unicausal]
6,MIR,78,8,0,MIR_78_8_0,"Elon Musk, the billionaire founder of Tesla In...","Elon Musk, the billionaire founder of Tesla In...",1,1,,1,"[Elon Musk, the billionaire founder of Tesla I...",1,[unicausal+]
7,MIR,94,1,0,MIR_94_1_0,The report’s focus solely on electric vehicles...,The report’s focus solely on electric vehicles...,1,1,,1,[The report’s focus solely on electric vehicle...,1,[unicausal+]
8,MIR,94,6,0,MIR_94_6_0,"To kick-start the shift, the report suggests b...","To kick-start the shift, <ARG1>the report sugg...",1,1,,1,"[To kick-start the shift, <ARG1>the report sug...",1,[unicausalm]
9,MIR,94,7,0,MIR_94_7_0,It also recommends setting up battery swapping...,It also recommends <ARG1>setting up battery sw...,1,1,,1,[It also recommends <ARG1>setting up battery s...,1,[unicausal]


In [7]:
print('Sentences:', len(grouped_pred['num_rs']))
print('Relations:', sum(grouped_pred['num_rs']))

Sentences: 42
Relations: 48


In [8]:
grouped_pred.to_csv(os.path.join(r'D:\66 CausalMap\Panasonic-IDS\outs','20230315_predictions.csv'), index=False, encoding='utf-8-sig')

# Get Predictions into JSON
as per CNC format

In [9]:
def get_text_w_pairs(text,cause,effect,allow_overlap=False):

    cause_start, cause_end = find_substring_in_text(text, cause)
    effect_start, effect_end = find_substring_in_text(text, effect)
    
    text_w_pairs = text
    if cause_start<effect_start: 
        if effect_start<cause_end:    # <ARG0><ARG1></ARG0>
            if not allow_overlap:
                return None
            effect_start+=6
        else:                         # <ARG0></ARG0><ARG1>
            effect_start+=6+7
        if effect_end<cause_end:      # <ARG0></ARG1></ARG0>
            if not allow_overlap:
                return None
            effect_end+=6
        else:                         # <ARG0></ARG0></ARG1>
            effect_end+=6+7
        text_w_pairs = text_w_pairs[:cause_start] + '<ARG0>' + text_w_pairs[cause_start:cause_end] + '</ARG0>'+ text_w_pairs[cause_end:]
        text_w_pairs = text_w_pairs[:effect_start] + '<ARG1>' + text_w_pairs[effect_start:effect_end] + '</ARG1>'+ text_w_pairs[effect_end:]
    else:
        if cause_start<effect_end:    # <ARG1><ARG0></ARG1>
            if not allow_overlap:
                return None
            cause_start+=6
        else:                         # <ARG1></ARG1><ARG0>
            cause_start+=6+7
        if cause_end<effect_end:      # <ARG1></ARG0></ARG1>
            if not allow_overlap:
                return None
            cause_end+=6
        else:                         # <ARG1></ARG1></ARG0>
            cause_end+=6+7
        text_w_pairs = text_w_pairs[:effect_start] + '<ARG1>' + text_w_pairs[effect_start:effect_end] + '</ARG1>'+ text_w_pairs[effect_end:]
        text_w_pairs = text_w_pairs[:cause_start] + '<ARG0>' + text_w_pairs[cause_start:cause_end] + '</ARG0>'+ text_w_pairs[cause_end:]
        
    if ('<ARG0>' not in text_w_pairs) or ('<ARG1>' not in text_w_pairs):
        raise ValueError(f'Cause/Effect Arg not found: \n{text_w_pairs}')
    
    return text_w_pairs


def ensure_prediction_tokens_tally(text, prediction):
    
    # ensure tokens tally with original text
    n_tokens_pred = len(prediction.split(' '))
    n_tokens = len(text.split(' '))
    
    if n_tokens_pred!=n_tokens:

        cause = re.search(r'<ARG0>(.*?)</ARG0>', prediction).group(1).strip()
        effect = re.search(r'<ARG1>(.*?)</ARG1>', prediction).group(1).strip()
        prediction = get_text_w_pairs(text,cause,effect,allow_overlap=True)
    
    # ensure ARG0 and ARG1 are not in the same token
    prediction = prediction.split(' ')
    final_prediction = []
    moving_token = None
    for tok in prediction:
        if moving_token is not None:
            tok = moving_token+tok
            moving_token=None
        
        if ('ARG0' in tok) and ('ARG1' in tok):
            # check which is the starting token, move it to the next item
            if '<ARG0>' in tok:
                moving_token='<ARG0>'
                tok = re.sub(moving_token,'',tok)
            else:
                moving_token='<ARG1>'
                tok = re.sub(moving_token,'',tok)
        
        final_prediction.append(tok)
        
    return ' '.join(final_prediction)

In [10]:
predictions = []
for i,row in annotated_df.iterrows():
    slice_df = grouped_pred[(grouped_pred['doc_id']==row.doc_id) & (grouped_pred['sent_id']==row.sent_id)]
    if len(slice_df)>0:
        # have predictions
        assert len(slice_df)==1
        preds = slice_df['causal_text_w_pairs'].iloc[0]
#         print(row.text)
#         print(preds)
#         print('\n')
        preds = [ensure_prediction_tokens_tally(row.text, p) for p in preds]
        predictions.append({"index": i, "prediction": preds})
    else:
        # no predictions
        predictions.append({"index": i, "prediction": [row.text]})

predictions[3]

{'index': 3,
 'prediction': ['Toyota in India has largely pivoted toward hybrid vehicles<ARG1>, which attract taxes of as much as 43% </ARG1>because <ARG0>they aren’t purely electric.</ARG0>Toyota, one of the world’s biggest carmakers, began operating in India in 1997.\r']}

In [11]:
import json

save_file_path = os.path.join(r'D:\66 CausalMap\Panasonic-IDS\outs','20230315_predictions.json')
with open(save_file_path, 'w') as fp:
    fp.write('\n'.join(json.dumps(i) for i in predictions))

In [12]:
#####################

# Evaluate JSON

In [13]:
import re
import json
import os
import sys
from ast import literal_eval
from datasets import load_metric
import pandas as pd
import itertools


def get_combinations(list1,list2):
    return [list(zip(each_permutation, list2)) for each_permutation in itertools.permutations(list1, len(list2))]


def read_predictions(submission_file):
    predictions = []
    with open(submission_file, "r") as reader:
        for line in reader:
            line = line.strip()
            if line:
                predictions.append(json.loads(line)['prediction'])
    return predictions


def clean_tok(tok):
    # Remove all other tags: E.g. <SIG0>, <SIG1>...
    return re.sub('</*[A-Z]+\d*>','',tok) 


def get_BIO(text_w_pairs):
    tokens = []
    ce_tags = []
    next_tag = tag = 'O'
    for tok in text_w_pairs.split(' '):

        # Replace if special
        if '<ARG0>' in tok:
            tok = re.sub('<ARG0>','',tok)
            tag = 'B-C'
            next_tag = 'I-C'
        elif '</ARG0>' in tok:
            tok = re.sub('</ARG0>','',tok)
            tag = 'I-C'
            next_tag = 'O'
        elif '<ARG1>' in tok:
            tok = re.sub('<ARG1>','',tok)
            tag = 'B-E'
            next_tag = 'I-E'
        elif '</ARG1>' in tok:
            tok = re.sub('</ARG1>','',tok)
            tag = 'I-E'
            next_tag = 'O'

        tokens.append(clean_tok(tok))
        ce_tags.append(tag)
        tag = next_tag
    
    return tokens, ce_tags


def get_BIO_all(text_w_pairs):
    tokens, ce_tags = get_BIO(text_w_pairs)
    assert(len(ce_tags)==len(tokens))
    return tokens, ce_tags


def format_results(ce_metric):
    final_results = {}
    metrics = ['precision','recall','f1','accuracy']
    final_results['Overall'] = {i:0 for i in metrics}

    results = ce_metric.compute()
    final_results['Cause'] = results['C']
    final_results['Effect'] = results['E']
    total_weight = 0
    for key in ['Cause','Effect']:
        ddict = final_results[key]
        for i in metrics[:-1]:
            final_results['Overall'][i]+=ddict[i]*ddict['number']
        total_weight+=ddict['number']
    for k,v in final_results['Overall'].items():
        final_results['Overall'][k]=v/total_weight
    final_results['Overall']['accuracy'] += results['overall_accuracy']*results['C']['number']
    accuracy_weight = results['C']['number']
    final_results['Overall']['accuracy'] /= accuracy_weight
    final_results['Overall']['number'] = accuracy_weight

    return final_results


def keep_best_combinations_only(row, refs, preds):
    final_results = {}
    best_metric = -1

    for points in get_combinations(row.id, row.id):
        
        # initialise
        ce_metric = load_metric('seqeval')

        # add rounds
        for a,b in list(points):
            _, ce_ref = refs[a]
            _, ce_pred = preds[b]
            ce_metric.add(
                prediction=ce_pred,
                reference=ce_ref 
            )
    
        # compute
        results = format_results(ce_metric)
        key_metric = float(results['Overall']['f1'])
        if key_metric>best_metric:
            # overwrite if best
            final_results=results
            best_metric=key_metric
            
    return final_results


def combine_dicts(d1,d2):
    the_keys = ['Cause','Effect']
    metrics = ['precision','recall','f1']    
    d0 = {k:{i:0 for i in metrics} for k in the_keys}
    
    for k in the_keys:
        total_weight=0
        if k in d1.keys():
            for m in metrics:
                d0[k][m]+=d1[k][m]*d1[k]['number']
            total_weight+=d1[k]['number']
        if k in d2.keys():
            for m in metrics:
                d0[k][m]+=d2[k][m]*d2[k]['number']
            total_weight+=d2[k]['number']
            
        for m in metrics:
            d0[k][m]/=total_weight
            d0[k]['number']=total_weight
    
    d0['Overall'] = {i:0 for i in metrics}
    total_weight = 0
    for key in the_keys:
        ddict = d0[key]
        for i in metrics:
            d0['Overall'][i]+=ddict[i]*ddict['number']
        total_weight+=ddict['number']
    for k,v in d0['Overall'].items():
        d0['Overall'][k]=v/total_weight
    
    d0['Overall']['number'] = d1['Overall']['number']+d2['Overall']['number']
    d0['Overall']['accuracy'] = d1['Overall']['accuracy']*d1['Overall']['number']
    d0['Overall']['accuracy'] += d2['Overall']['accuracy']*d2['Overall']['number']
    d0['Overall']['accuracy'] /= d0['Overall']['number']
    
    return d0


def main(ref_df, pred_list, calculate_best_combi=True):
    # Convert
    ref_df, pred_list = keep_relevant_rows_and_unstack(ref_df, pred_list)
    assert(len(pred_list)==len(ref_df))
    refs = [get_BIO_all(i) for i in ref_df['text_w_pairs']]
    preds = [get_BIO_all(i) for i in pred_list]
    
    # Group
    if calculate_best_combi:
        grouped_df = ref_df.copy()
        grouped_df['id'] = [[i] for i in grouped_df.index]
        grouped_df = grouped_df.groupby(['corpus','doc_id','sent_id'])[['eg_id','id']].agg({'eg_id':'count','id':'sum'}).reset_index()
        grouped_df = grouped_df[grouped_df['eg_id']>1]
        req_combi_ids = [item for sublist in grouped_df['id'] for item in sublist]
    else:
        grouped_df = None
        req_combi_ids = []

    # For examples that DO NOT require combination search
    regular_ids = list(set(range(len(preds)))-set(req_combi_ids))

    ce_metric = load_metric('seqeval')

    for i in regular_ids:
        _, ce_ref = refs[i]
        _, ce_pred = preds[i]
        ce_metric.add(
            prediction=ce_pred,
            reference=ce_ref 
        )
    final_result = format_results(ce_metric)

    if grouped_df is not None:
        # For examples that require combination search
        for _, row in grouped_df.iterrows():
            best_results = keep_best_combinations_only(row, refs, preds)
            final_result = combine_dicts(final_result,best_results)

    return final_result
    

def keep_relevant_rows_and_unstack(ref_df, predictions):
    
    # Keep only causal examples
    predictions_w_true_labels = []
    eg_id_counter = []
    for i, row in ref_df.iterrows():
        if row.num_rs>0:
            p = predictions[i]
            if len(p)>row.num_rs:
                # Note if you predict more than the number of relations we have, we only keep the first few.
                # We completely ignore the subsequent predictions.
                p = p[:row.num_rs]
            elif len(p)<row.num_rs:
                # Incorporate dummy predictions if there are insufficient predictions
                p.extend([row.text]*(row.num_rs-len(p)))
            predictions_w_true_labels.extend(p)
            eg_id_counter.extend(list(range(row.num_rs)))
    ref_df = ref_df[ref_df['num_rs']>0].reset_index(drop=True)
    
    # Expand into single rows
    ref_df = ref_df.drop(['text_w_pairs'], axis=1)
    ref_df['causal_text_w_pairs'] = ref_df['causal_text_w_pairs'].apply(lambda x: literal_eval(x))
    ref_df = ref_df.explode('causal_text_w_pairs')
    ref_df = ref_df.rename(columns={'causal_text_w_pairs':'text_w_pairs'})
    ref_df['eg_id'] = eg_id_counter
    
    return ref_df.reset_index(drop=True), predictions_w_true_labels

In [14]:
# set save files
output_filename = os.path.join(r"D:\66 CausalMap\Panasonic-IDS\outs", '20230315_scores.txt')
output_file = open(output_filename, 'w')

# read files
truth_file = r"D:\66 CausalMap\Panasonic-IDS\data\MIR_annotated_grouped.csv"
truth = pd.read_csv(truth_file, encoding="utf-8")
submission_answer_file = r"D:\66 CausalMap\Panasonic-IDS\outs\20230315_predictions.json"
predictions = read_predictions(submission_answer_file)

if len(truth) != len(predictions):
    raise IndexError("Number of entries in the submission.json do not match with the ground truth entry count!")
else:
    # evaluate
    final_results = main(truth, predictions, calculate_best_combi=True)
    # write to output file
    level_order = ['Overall','Cause','Effect']
    columns_order = ['Recall', 'Precision', 'F1', 'Accuracy', 'Number']

    for level in level_order:
        ddict = final_results[level]
        for k in columns_order:
            if k.lower() in ddict:
                v = ddict[k.lower()]
                if level=='Overall':
                    output_file.write("{0}:{1}\n".format(k.title(),v))
                else:
                    output_file.write("{0}_{1}:{2}\n".format(level,k.title(),v))
            else:
                continue
    # output_file.write("{0}:{1}\n".format("MCC",0))

output_file.close()

ValueError: Found input variables with inconsistent numbers of samples:
[28, 27]
[27, 27]